## Python3 - XML
`xml.etree.ElementTree`

---
`xml.etree.ElementTree` 模块提供了简单有效的解析和生成 `XML` 数据的 `API` 接口
* `XML` 树和元素
    `XML` 是一种天然的分层数据表示，通常用来表示一棵树，在整个模块中，`ElementTree` 表示整个树，`Element` 表示节点，和整个文档的交互工作基本上都是依赖的 `ElementTree`,对文档中的元素的交互一般都是依靠 `Element` 元素
    测试文档如下
    ```xml
    <?xml version="1.0"?>
<data>
    <country name="Liechtenstein">
        <rank>1</rank>
        <year>2008</year>
        <gdppc>141100</gdppc>
        <neighbor name="Austria" direction="E"/>
        <neighbor name="Switzerland" direction="W"/>
    </country>
    <country name="Singapore">
        <rank>4</rank>
        <year>2011</year>
        <gdppc>59900</gdppc>
        <neighbor name="Malaysia" direction="N"/>
    </country>
    <country name="Panama">
        <rank>68</rank>
        <year>2011</year>
        <gdppc>13600</gdppc>
        <neighbor name="Costa Rica" direction="W"/>
        <neighbor name="Colombia" direction="E"/>
    </country>
</data>
    ```
* 导入包

In [2]:
import xml.etree.ElementTree as ET

* 导入数据
    1. `ET.parse(filename)`: 从文件中解析获得 `ElementTree`
    2. `tree.getroot()`: 从 `ElementTree` 中获得 `root` 节点(`ELement`)
    3. `ET.fromstring(data)`: 直接从字符串中解析 `XML` 文件，返回 `root, Element`
    4. 还存在很多的解析的函数，返回的结果各不相同，可以通过查看官方文档获得详细的返回信息

In [3]:
# tree = ET.parse("filename.xml")
data = '''<?xml version="1.0"?>
<data>
    <country name="Liechtenstein">
        <rank>1</rank>
        <year>2008</year>
        <gdppc>141100</gdppc>
        <neighbor name="Austria" direction="E"/>
        <neighbor name="Switzerland" direction="W"/>
    </country>
    <country name="Singapore">
        <rank>4</rank>
        <year>2011</year>
        <gdppc>59900</gdppc>
        <neighbor name="Malaysia" direction="N"/>
    </country>
    <country name="Panama">
        <rank>68</rank>
        <year>2011</year>
        <gdppc>13600</gdppc>
        <neighbor name="Costa Rica" direction="W"/>
        <neighbor name="Colombia" direction="E"/>
    </country>
</data>
'''

root = ET.fromstring(data)
root

<Element 'data' at 0x7f04373ed778>

* 文档树详细信息
    1. 每一个 `Element` 节点都存在有对应的属性
        * `tag`: 标签信息
        * `attrib`: 属性信息,标签内部的属性信息可以通过 `Element.get("attrib name")` 的方式获得
        * `text`: 文本信息
    2. 迭代查看所有的子节点，因为子节点是嵌套的，所以可以采用索引的方式找到对应的子节点
    3. 使用函数 `ET.dump(root)` 可以打印从 `root` 开始的子树，无返回值
        只可以打印 `tree`, 不可以打印 `Element`
    4. 排版打印出文档树
        ```python
        from lxml import etree
        root = tree.getroot()
        res = ET.tostring(root)
        print(etree.tostring(etree.fromstring(res), pretty_print=True))
        ```

In [4]:
root.tag, root.attrib, root.text

('data', {}, '\n    ')

In [5]:
# 迭代查看所有的子节点
for child in root:
    print(child.tag, child.attrib)

# 索引, 标签　Singapore -> year
print(root[1][1].text)

country {'name': 'Liechtenstein'}
country {'name': 'Singapore'}
country {'name': 'Panama'}
2011


* 非阻塞解析  
  到目前为止，模块中的所有的提供的解析函数要求必须输入整个文档来解析结果，但是可以通过使用 `XMLParser` 每次递增的喂给它数据实现动态解析

In [6]:
parser = ET.XMLPullParser()
parser.feed('<data name="lantian">')
parser.feed('</data>')
for i in parser.read_events():
    print(i[1].tag, i[1].attrib)

data {'name': 'lantian'}


* 寻找节点  
    `Element` 元素有一些很实用的方法辅助查询子树下的元素  
    1. `Element.iter("tag name")`: 在子树下查询所有的标签是 `tag_name` 的 `Element`,如果为空表示全部子树节点都查询
    2. `Element.findall("tag name")`: 只找直接孩子节点
    3. `Element.find("tag name")`: 只找第一个找到的孩子节点，单返回

In [7]:
# Element.iter
for sub in root.iter("neighbor"):
    print(sub.tag, sub.attrib)

print("-" * 50)
# 
for country in root.findall('country'):
    rank = country.find('rank').text
    name = country.get('name')
    print(name, rank)

neighbor {'direction': 'E', 'name': 'Austria'}
neighbor {'direction': 'W', 'name': 'Switzerland'}
neighbor {'direction': 'N', 'name': 'Malaysia'}
neighbor {'direction': 'W', 'name': 'Costa Rica'}
neighbor {'direction': 'E', 'name': 'Colombia'}
--------------------------------------------------
Liechtenstein 1
Singapore 4
Panama 68


* 修改 `XML` 文件  
    `ElementTree` 模块提供基本的方法帮助构建 `XML` 文件并将其写入文件, `ElementTree.write()`
    1. 修改 `Element` 节点的基本方法
        * text: text文本域可以直接的修改
        * 添加节点的属性: 可以通过 `Element.set("attrib_name", "attrib_value")` 实现
        * 添加新的子节点 `Element.append(Element)`: 参数是 `Element` 对象
        * 移除节点 `Element.remove(Element)`: 参数是 `Element` 对象
    2. 手动构建 `ElementTree`
        ```python
        a = ET.Element('a')
        b = ET.SubElement(a, 'b')    # b 是 a 的直接
        ```
   
* 构建 `XML` 文件
    

In [9]:
# 删除 country - Singapore 标签
for country in root.findall("country"):
    if country.attrib['name'] == 'Singapore':
        root.remove(country)
ET.dump(root)

print('-' * 50)

# Element.append
root.append(ET.Element('a'))
ET.dump(root)

print('-' * 50)

# 构建文档树
a = ET.Element('a')
b = ET.SubElement(a, 'b')
c = ET.SubElement(a, 'c')
d = ET.SubElement(c, 'd')
d.text = "lantian"
ET.dump(a)

<data>
    <country name="Liechtenstein">
        <rank>1</rank>
        <year>2008</year>
        <gdppc>141100</gdppc>
        <neighbor direction="E" name="Austria" />
        <neighbor direction="W" name="Switzerland" />
    </country>
    <country name="Panama">
        <rank>68</rank>
        <year>2011</year>
        <gdppc>13600</gdppc>
        <neighbor direction="W" name="Costa Rica" />
        <neighbor direction="E" name="Colombia" />
    </country>
<a /></data>
--------------------------------------------------
<data>
    <country name="Liechtenstein">
        <rank>1</rank>
        <year>2008</year>
        <gdppc>141100</gdppc>
        <neighbor direction="E" name="Austria" />
        <neighbor direction="W" name="Switzerland" />
    </country>
    <country name="Panama">
        <rank>68</rank>
        <year>2011</year>
        <gdppc>13600</gdppc>
        <neighbor direction="W" name="Costa Rica" />
        <neighbor direction="E" name="Colombia" />
    </country>
<a /